## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV
import joblib
from time import time
import warnings
warnings.filterwarnings('ignore')

## Chargement des donnees preprocessees

In [2]:
X_train = np.load('data/X_train_scaled.npy')
X_test = np.load('data/X_test_scaled.npy')
y_train = np.load('data/y_train.npy')
y_test = np.load('data/y_test.npy')
le = joblib.load('models/label_encoder.pkl')

print(" Donnees chargees:")
print("="*60)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape:  {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape:  {y_test.shape}")
print(f"\nNombre de classes: {len(le.classes_)}")
print(f"Classes: {le.classes_}")

## Hyperparameter Tuning - Grilles essentielles

In [3]:
n_samples = len(X_train)
max_k = int(np.sqrt(n_samples))
if max_k % 2 == 0:
    max_k -= 1
k_values = list(range(3, max_k + 1, 2))

param_grids = {
    'KNN': {
        'n_neighbors': k_values,
        'weights': ['uniform', 'distance']
    },
    
    'Decision Tree': {
        'max_depth': [5, 10, 15, 20, None],
        'min_samples_split': [2, 5, 10],
        'criterion': ['gini', 'entropy']
    },
    
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'max_features': ['sqrt', 'log2']
    },
    
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 0.001, 0.01, 0.1],
        'kernel': ['rbf']
    },
    
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100]
    },
    
    'Neural Network': {
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (150, 100, 50)],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive']
    },
    
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7]
    }
}

print("Grilles d'hyperparametres definies")
for model_name, params in param_grids.items():
    n_combinations = 1
    for param_values in params.values():
        n_combinations *= len(param_values)
    print(f"{model_name:25s} -> {len(params):2d} parametres, {n_combinations:,} combinaisons")

## Definition et optimisation des modeles

In [4]:
base_models = {
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'Naive Bayes': GaussianNB(),
    'SVM': SVC(probability=True, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=2000, random_state=42, n_jobs=-1),
    'Neural Network': MLPClassifier(max_iter=1000, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis()
}

cv_folds = 10
cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

models_grid = ['KNN', 'Logistic Regression']
models_random = ['Decision Tree', 'Random Forest', 'SVM', 'Neural Network', 'Gradient Boosting']
models_default = ['LDA', 'QDA', 'Naive Bayes']

print(f"Configuration CV: {cv_folds}-fold Stratified")
print(f"\nModeles pour GridSearchCV: {len(models_grid)}")
print(f"Modeles pour RandomizedSearchCV: {len(models_random)}")
print(f"Modeles sans optimisation (par defaut): {len(models_default)}")

## Optimisation GridSearchCV

In [5]:
grid_results = {}
best_models = {}

print("DEBUT GRIDSEARCHCV")
print("="*70)

for i, model_name in enumerate(models_grid, 1):
    print(f"\n[{i}/{len(models_grid)}] {model_name}")
    print("-"*70)
    
    start_time = time()
    
    grid_search = GridSearchCV(
        estimator=base_models[model_name],
        param_grid=param_grids[model_name],
        cv=cv,
        scoring='accuracy',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train, y_train)
    
    elapsed = time() - start_time
    
    best_cv = grid_search.best_score_
    test_score = grid_search.score(X_test, y_test)
    
    grid_results[model_name] = {
        'best_params': grid_search.best_params_,
        'cv_score': best_cv,
        'test_score': test_score,
        'time': elapsed
    }
    
    best_models[model_name] = grid_search.best_estimator_
    
    print(f"Meilleur CV: {best_cv:.4f}")
    print(f"Test Acc: {test_score:.4f}")
    print(f"Temps: {elapsed:.2f}s")
    print(f"Params: {grid_search.best_params_}")

print("\n" + "="*70)
print("GRIDSEARCHCV TERMINE")
print("="*70)

## Optimisation RandomizedSearchCV

In [6]:
n_iter = 50

print("DEBUT RANDOMIZEDSEARCHCV")
print("="*70)
print(f"Iterations par modele: {n_iter}\n")

for i, model_name in enumerate(models_random, 1):
    print(f"\n[{i}/{len(models_random)}] {model_name}")
    print("-"*70)
    
    start_time = time()
    
    random_search = RandomizedSearchCV(
        estimator=base_models[model_name],
        param_distributions=param_grids[model_name],
        n_iter=n_iter,
        cv=cv,
        scoring='accuracy',
        n_jobs=-1,
        verbose=0,
        random_state=42
    )
    
    random_search.fit(X_train, y_train)
    
    elapsed = time() - start_time
    
    best_cv = random_search.best_score_
    test_score = random_search.score(X_test, y_test)
    
    grid_results[model_name] = {
        'best_params': random_search.best_params_,
        'cv_score': best_cv,
        'test_score': test_score,
        'time': elapsed
    }
    
    best_models[model_name] = random_search.best_estimator_
    
    print(f"Meilleur CV: {best_cv:.4f}")
    print(f"Test Acc: {test_score:.4f}")
    print(f"Temps: {elapsed:.2f}s")
    print(f"Params: {random_search.best_params_}")

print("\n" + "="*70)
print("RANDOMIZEDSEARCHCV TERMINE")
print("="*70)

## Entrainement modeles sans optimisation

In [7]:
print("\nDEBUT ENTRAINEMENT MODELES PAR DEFAUT")
print("="*70)
print(f"Modeles: {', '.join(models_default)}\n")

for i, model_name in enumerate(models_default, 1):
    print(f"\n[{i}/{len(models_default)}] {model_name}")
    print("-"*70)
    
    start_time = time()
    
    model = base_models[model_name]
    model.fit(X_train, y_train)
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
    
    elapsed = time() - start_time
    
    best_cv = cv_scores.mean()
    test_score = model.score(X_test, y_test)
    
    grid_results[model_name] = {
        'best_params': {},
        'cv_score': best_cv,
        'test_score': test_score,
        'time': elapsed
    }
    
    best_models[model_name] = model
    
    print(f"CV Score: {best_cv:.4f}")
    print(f"Test Acc: {test_score:.4f}")
    print(f"Temps: {elapsed:.2f}s")
    print(f"Params: Par defaut")

print("\n" + "="*70)
print("ENTRAINEMENT PAR DEFAUT TERMINE")
print("="*70)

## Evaluation finale des modeles optimises

In [8]:
results = []
trained_models = {}

print("\nEVALUATION FINALE")
print("="*70)

for model_name, model in best_models.items():
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    cv_score = grid_results[model_name]['cv_score']
    
    results.append({
        'Model': model_name,
        'CV Score': cv_score,
        'Train Accuracy': train_score,
        'Test Accuracy': test_score,
        'Training Time (s)': grid_results[model_name]['time']
    })
    
    trained_models[model_name] = {
        'model': model,
        'y_train_pred': y_train_pred,
        'y_test_pred': y_test_pred,
        'best_params': grid_results[model_name]['best_params']
    }

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Test Accuracy', ascending=False)
results_df['Rank'] = range(1, len(results_df) + 1)

results_df = results_df[[
    'Rank', 'Model', 'CV Score', 'Train Accuracy', 'Test Accuracy', 'Training Time (s)'
]]

print("\nRESULTATS (tries par Test Accuracy):")
print("="*140)
print(results_df.to_string(index=False))

print(f"\nMEILLEUR MODELE:")
print("="*60)
print(f"{results_df.iloc[0]['Model']}: {results_df.iloc[0]['Test Accuracy']:.4f}")

# Tableau récapitulatif

In [9]:
from sklearn.metrics import f1_score
f1_rows = []

for model_name, info in trained_models.items():
    y_train_pred = info["y_train_pred"]
    y_test_pred  = info["y_test_pred"]
    
    f1_train = f1_score(y_train, y_train_pred, average="macro")
    f1_test  = f1_score(y_test, y_test_pred,  average="macro")
    
    f1_rows.append({
        "Model": model_name,
        "Train F1_macro": f1_train,
        "Test F1_macro":  f1_test
    })

f1_df = pd.DataFrame(f1_rows)

# Fusion avec ton tableau des résultats
results_df = results_df.merge(f1_df, on="Model")

# Pour la sélection, on va utiliser le F1 sur le test → on renomme pour faire simple
results_df = results_df.rename(columns={"Test F1_macro": "F1_macro"})

results_df


# Barplot des 10 modèles

In [10]:
df_plot = results_df.sort_values("F1_macro", ascending=False)

models = df_plot["Model"]
accuracy = df_plot["Test Accuracy"]
f1_scores = df_plot["F1_macro"]

x = np.arange(len(models))
width = 0.35

plt.figure(figsize=(12, 6))

# 🎨 Couleurs personnalisées
dark_green = "#1B5E20"     # Vert foncé
light_green = "#66BB6A"    # Vert clair

# Barres côte à côte
bars_acc = plt.bar(x - width/2, accuracy, width, label="Accuracy (test)",
                   color=dark_green, edgecolor="black", linewidth=0.6)

bars_f1 = plt.bar(x + width/2, f1_scores, width, label="F1-macro (test)",
                  color=light_green, edgecolor="black", linewidth=0.6)

# ➕ Ajouter les valeurs au-dessus des barres
for bar in bars_acc:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.0002,
             f"{height:.4f}", ha="center", va="bottom", fontsize=8)

for bar in bars_f1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.0002,
             f"{height:.4f}", ha="center", va="bottom", fontsize=8)

plt.title("Comparaison des modèles (Accuracy & F1-macro) après optimisation", fontsize=14)
plt.ylabel("Score", fontsize=12)
plt.xticks(x, models, rotation=45, ha="right")

plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()


# Graphique Overfitting (Train vs Test)

In [11]:
df_plot = results_df.copy()

models = df_plot["Model"]
train_acc = df_plot["Train Accuracy"]
test_acc = df_plot["Test Accuracy"]
train_f1 = df_plot["Train F1_macro"]
test_f1 = df_plot["F1_macro"]   # test F1 renommé

x = np.arange(len(models))
width = 0.2

plt.figure(figsize=(16, 6))

# 🎨 Couleurs personnalisées
dark_green = "#1B5E20"     # Vert foncé
light_green = "#66BB6A"    # Vert clair

plt.subplot(1, 2, 1)
bars1 = plt.bar(x - width, train_acc, width, label="Train Accuracy",
                color=dark_green, edgecolor="black")
bars2 = plt.bar(x + width, test_acc, width, label="Test Accuracy",
                color=light_green, edgecolor="black")

plt.title("Train vs Test Accuracy (Détection d'Overfitting)", fontsize=13)
plt.xticks(x, models, rotation=45, ha="right")
plt.ylabel("Accuracy")

# ➕ Afficher les valeurs au-dessus des barres
for bar in bars1:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f"{bar.get_height():.3f}", ha="center", va="bottom", fontsize=9)

for bar in bars2:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f"{bar.get_height():.3f}", ha="center", va="bottom", fontsize=9)

plt.legend()
plt.subplot(1, 2, 2)
bars3 = plt.bar(x - width, train_f1, width, label="Train F1_macro",
                color=dark_green, edgecolor="black")
bars4 = plt.bar(x + width, test_f1, width, label="Test F1_macro",
                color=light_green, edgecolor="black")

plt.title("Train vs Test F1_macro (Détection d'Overfitting)", fontsize=13)
plt.xticks(x, models, rotation=45, ha="right")
plt.ylabel("F1-macro")

for bar in bars3:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f"{bar.get_height():.3f}", ha="center", va="bottom", fontsize=9)

for bar in bars4:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f"{bar.get_height():.3f}", ha="center", va="bottom", fontsize=9)

plt.legend()

plt.tight_layout()
plt.show()


In [12]:
table_overfit = pd.DataFrame({
    "Modèle": results_df["Model"],
    "Train Accuracy": results_df["Train Accuracy"].round(4),
    "Test Accuracy": results_df["Test Accuracy"].round(4),
    "Train F1_macro": results_df["Train F1_macro"].round(4),
    "Test F1_macro": results_df["F1_macro"].round(4)
})

# Calcul de la différence pour montrer qu'il n'y a PAS d’overfitting
table_overfit["Diff Accuracy"] = (table_overfit["Train Accuracy"] - table_overfit["Test Accuracy"]).abs().round(4)
table_overfit["Diff F1"] = (table_overfit["Train F1_macro"] - table_overfit["Test F1_macro"]).abs().round(4)

# Affichage
table_overfit.style.background_gradient(cmap="Greens").set_caption("Analyse Overfitting : Différences Train vs Test")

# Selection Top 3

In [13]:
results_df["Score_global"] = (
    results_df["Test Accuracy"] + results_df["F1_macro"]
) / 2

# Tri + sélection des 3 meilleurs
top3_df = (
    results_df
    .sort_values(["Score_global", "Test Accuracy", "F1_macro"], ascending=False)
    .head(3)
)

top3_df[[
    "Rank", "Model", "CV Score", "Test Accuracy", "F1_macro", "Score_global"
]]

# Hyperparamètres Top 3

In [14]:
param_grids_top3 = {
    "Naive Bayes": {
        # Très peu de paramètres → GridSearch très rapide
        "var_smoothing": [1e-9, 1e-8, 1e-7]
    },
    
    "Random Forest": {
    "n_estimators": [100, 300, 500],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt", "log2"]
    },

    "Gradient Boosting": {
        # Grille compacte mais utile
        "n_estimators": [100, 200,500],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5]
    }
}

param_grids_top3

# GridSearchCV sur les 3 Top modèles

In [15]:
best_top3 = []

for model_name in top3_df["Model"]:
    print(f"\n🔍 GridSearchCV pour : {model_name}")
    
    # modèle de base (déjà défini plus haut dans ton notebook)
    base_model = base_models[model_name]
    
    # grille d'hyperparamètres spécifique au modèle
    grid_params = param_grids_top3[model_name]
    
    grid = GridSearchCV(
        estimator=base_model,
        param_grid=grid_params,
        scoring="f1_macro",   # tu peux mettre "accuracy" si tu veux tester
        cv=3,                 # 3 folds seulement pour réduire le temps
        n_jobs=-1             # utilise tous les CPU dispo
    )
    
    grid.fit(X_train, y_train)
    
    best_top3.append({
        "Model": model_name,
        "Best Estimator": grid.best_estimator_,
        "Best Params": grid.best_params_,
        "Best CV Score (F1_macro)": grid.best_score_
    })

import pandas as pd
best_top3_df = pd.DataFrame(best_top3).sort_values(
    "Best CV Score (F1_macro)", ascending=False
)

best_top3_df

# Afficher le Top 1 final

In [16]:
best_model_row = best_top3_df.iloc[0]
best_model = best_model_row["Best Estimator"]

# Prédictions sur le jeu de test
y_pred_test = best_model.predict(X_test)

# Mesures sur TEST
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1_macro = f1_score(y_test, y_pred_test, average="macro")

print("\nMEILLEUR MODELE FINAL APRES GRIDSEARCH SUR LE TOP 3")
print("=" * 70)
print(f"Modèle                 : {best_model_row['Model']}")
print(f"Best CV F1_macro       : {best_model_row['Best CV Score (F1_macro)']:.4f}")
print(f"Test Accuracy          : {test_accuracy:.4f}")
print(f"F1_macro (test)        : {test_f1_macro:.4f}")

print("\nMeilleurs hyperparamètres :")
print(best_model_row["Best Params"])

## Sauvegarde des modeles

In [17]:
import os
import json

os.makedirs('models/tuned', exist_ok=True)
os.makedirs('results/tuning', exist_ok=True)

print("\nSauvegarde du meilleur modele:")
print("="*60)

best_model_name = results_df.iloc[0]['Model']
best_model_data = trained_models[best_model_name]

model_filename = f"models/tuned/{best_model_name.replace(' ', '_').lower()}_best.pkl"
joblib.dump(best_model_data['model'], model_filename)
print(f"Modele: {model_filename}")

best_model_info = {
    'model_name': best_model_name,
    'best_params': grid_results[best_model_name]['best_params'],
    'cv_score': float(grid_results[best_model_name]['cv_score']),
    'test_score': float(grid_results[best_model_name]['test_score']),
    'train_score': float(results_df.iloc[0]['Train Accuracy']),
    'time': float(grid_results[best_model_name]['time'])
}

with open('results/tuning/best_model_info.json', 'w', encoding='utf-8') as f:
    json.dump(best_model_info, f, indent=4, ensure_ascii=False)

results_df.to_csv('results/tuning/tuning_results.csv', index=False)

all_models_performance = []
for model_name in best_models.keys():
    all_models_performance.append({
        'Model': model_name,
        'CV Score': grid_results[model_name]['cv_score'],
        'Train Accuracy': results_df[results_df['Model'] == model_name]['Train Accuracy'].values[0],
        'Test Accuracy': results_df[results_df['Model'] == model_name]['Test Accuracy'].values[0],
        'Training Time (s)': grid_results[model_name]['time'],
        'Best Params': str(grid_results[model_name]['best_params'])
    })

performance_df = pd.DataFrame(all_models_performance)
performance_df = performance_df.sort_values('Test Accuracy', ascending=False)
performance_df.to_csv('results/tuning/all_models_performance.csv', index=False)

print("\nFichiers sauvegardes:")
print(f"  - {model_filename}")
print("  - results/tuning/best_model_info.json")
print("  - results/tuning/tuning_results.csv")
print("  - results/tuning/all_models_performance.csv")

## Visualisation des resultats

In [18]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(18, 6), facecolor='white')

y_pos = np.arange(len(results_df))
axes[0].barh(y_pos, results_df['Test Accuracy'], 
             color='skyblue', edgecolor='black', linewidth=1.2)
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels(results_df['Model'])
axes[0].set_xlabel('Test Accuracy', fontweight='bold', fontsize=12)
axes[0].set_title('Test Accuracy par Modele', fontsize=14, fontweight='bold')
axes[0].set_xlim([0, 1])
axes[0].grid(axis='x', alpha=0.3)

for i, acc in enumerate(results_df['Test Accuracy']):
    axes[0].text(acc + 0.01, i, f'{acc:.3f}', va='center', fontsize=10)

x_pos = np.arange(len(results_df))
axes[1].bar(x_pos, results_df['CV Score'],
            color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[1].set_ylabel('CV Accuracy', fontweight='bold', fontsize=12)
axes[1].set_title('Cross-Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_ylim([0, 1.1])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/visualizations/training_results.png', dpi=300, bbox_inches='tight')
plt.close()

from IPython.display import Image, display
display(Image('results/visualizations/training_results.png'))

print("\nGraphique sauvegarde: results/visualizations/training_results.png")

## Resume final

In [19]:
print(f"\n" + "="*70)
print(" RESUME DE L'ENTRAINEMENT")
print("="*70)
print(f"""
Modeles entraines: {len(best_models)}
Cross-Validation: {cv_folds}-fold Stratified
Train set: {X_train.shape[0]} echantillons
Test set: {X_test.shape[0]} echantillons

Meilleur modele (Test Accuracy):
  {results_df.iloc[0]['Model']} -> {results_df.iloc[0]['Test Accuracy']:.4f}""")

print("\nMeilleurs hyperparametres:")
best_model_name = results_df.iloc[0]['Model']
best_params = trained_models[best_model_name]['best_params']
for param, value in best_params.items():
    print(f"  {param}: {value}")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=37cd5642-eb00-4cb1-969e-a9bc85cf5e83' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>